In [1]:
import pandas as pd 
import nfl_data_py as nfl

In [2]:
pd.set_option('display.max_columns', None)  # Keine Begrenzung für Spaltenanzahl
pd.set_option('display.width', None)       # Keine Begrenzung der Breite

In [6]:
from data_loading import load_data, prepare_features

In [7]:
df = load_data()
df = prepare_features(df)

Downcasting floats.


/Users/simonmuhlbauer/Desktop/Uni/Master/WiSe_24-25/DSAI/Projekt/ff_data_science/src/data_loading.py:157: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1).ewm(span=5, min_periods=5).mean())
/Users/simonmuhlbauer/Desktop/Uni/Master/WiSe_24-25/DSAI/Projekt/ff_data_science/src/data_loading.py:171: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30182 entries, 0 to 30181
Data columns (total 60 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   player_id                             30182 non-null  int64  
 1   season                                30182 non-null  int32  
 2   week                                  30182 non-null  int32  
 3   fantasy_points                        30182 non-null  float32
 4   draft_pick                            30182 non-null  float64
 5   dom                                   30182 non-null  float64
 6   rookie_flag                           30182 non-null  int64  
 7   home                                  30182 non-null  int64  
 8   ewm_opponent_team_points_allowed_l5w  30182 non-null  float64
 9   min_opponent_team_points_allowed_l3w  30182 non-null  float64
 10  max_opponent_team_points_allowed_l3w  30182 non-null  float64
 11  ewm_recent_team